<a href="https://colab.research.google.com/github/thainta/zaloAI/blob/main/ZALO_AI_TASK1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GET DATA

In [ ]:
! wget https://dl.challenge.zalo.ai/news-summarization/data/train.zip
! wget https://dl.challenge.zalo.ai/news-summarization/data/public_test.zip 
! wget https://dl.challenge.zalo.ai/news-summarization/data/public_test_sample_submit.zip

--2020-11-14 04:13:55--  https://dl.challenge.zalo.ai/news-summarization/data/train.zip
Resolving dl.challenge.zalo.ai (dl.challenge.zalo.ai)... 49.213.78.231
Connecting to dl.challenge.zalo.ai (dl.challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1224058 (1.2M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   1.17M  1.42MB/s    in 0.8s    

2020-11-14 04:13:57 (1.42 MB/s) - ‘train.zip’ saved [1224058/1224058]

--2020-11-14 04:13:57--  https://dl.challenge.zalo.ai/news-summarization/data/public_test.zip
Resolving dl.challenge.zalo.ai (dl.challenge.zalo.ai)... 49.213.78.231
Connecting to dl.challenge.zalo.ai (dl.challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 319489 (312K) [application/zip]
Saving to: ‘public_test.zip’

public_test.zip     100%[===================>] 312.00K   627KB/s    in 0.5s    

2020-11-14 04:13:58 (627 KB/s) -

In [ ]:
! unzip public_test.zip
! unzip public_test_sample_submit.zip
! unzip train.zip

Archive:  public_test.zip
   creating: public_test/public_test/
  inflating: public_test/public_test/public_test.jsonl  
 extracting: public_test/public_test/export_date.txt  
Archive:  public_test_sample_submit.zip
   creating: public_test/public_test_sample_submit/
  inflating: public_test/public_test_sample_submit/public_test_sample_submit.jsonl  
Archive:  train.zip
   creating: train/
  inflating: train/train.jsonl       
 extracting: train/export_date.txt   


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
train = pd.read_json("/content/train/train.jsonl", lines = True)
test  = pd.read_json("/content/public_test/public_test/public_test.jsonl", lines = True)
public_test_sample = pd.read_json("/content/public_test/public_test_sample_submit/public_test_sample_submit.jsonl",lines = True)

# TRAIN NAME

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 168 µs


In [ ]:
!pip install vncorenlp
! wget https://github.com/vncorenlp/VnCoreNLP/archive/master.zip
! wget http://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html
! unzip master.zip

     |████████████████████████████████| 2.7MB 8.7MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=b795ec056778ee1e35efebfadf12d720dc64867d9030c265504a0b93afe1d15a
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp
--2020-11-14 04:20:19--  https://github.com/vncorenlp/VnCoreNLP/archive/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vncorenlp/VnCoreNLP/zip/master [following]
--2020-11-14 04:20:20--  https://codeload.github.com/vncorenlp/VnCoreNLP/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [

IMPORT LIB

In [ ]:
from vncorenlp import VnCoreNLP
from collections import Counter
import pprint as pp
import json

time: 71 ms


In [ ]:
vncorenlp_file = '/content/VnCoreNLP-master/VnCoreNLP-1.1.1.jar'

time: 1.03 ms


TRAIN

In [ ]:
list_dict = []
for p in range(19,20): 
  text = test['original_doc'][p]
  test_id = test['test_id'][p]
  size_temp = len(text["_source"]["body"])
  sentences = [] 
  for p in range(size_temp):
    temp = ((text["_source"]["body"][p]["text"]))
    sentences.append(temp)
  s = str()
  s = s + ' ' + ' '.join(sentences)
  s = s + '. ' + str(text["_source"]["description"])
  clean_text = s
  scores =  []
  for j in range(len(clean_text)):
    if clean_text[j] == '-' and clean_text[j+2]!='-' and clean_text[j-2]!='-':
      if clean_text[j-1].isnumeric() and clean_text[j+1].isnumeric():
        scores.append(clean_text[j-1]+clean_text[j]+clean_text[j+1])
    if clean_text[j] == '-' and clean_text[j-2].isnumeric() and clean_text[j+2].isnumeric():
      scores.append(clean_text[j-2]+clean_text[j]+clean_text[j+2])
  listA = scores
  scores_occurence_count = Counter(listA)
  final_score_1=scores_occurence_count.most_common(1)[0][0][0]
  final_score_2=scores_occurence_count.most_common(1)[0][0][2]
  #print(clean_text)
  with VnCoreNLP(vncorenlp_file) as vncorenlp:
    ner = list(vncorenlp.ner(clean_text))
  print(ner)
  ner_merge = sum(ner,[])
  soccer_team = [list(x) for x in ner_merge if x[1] == 'B-ORG' or x[1] =='I-ORG' or x[1] =='B-LOC' or x[1] =='I-LOC']
  i = 0
  while i< (len(soccer_team)):
    if i == len(soccer_team) - 1 and soccer_team[i][1] != 'I-LOC' and soccer_team[i][1] != 'I-ORG':
      break
    if soccer_team[i][1] == 'B-ORG' and soccer_team[i+1][1] == 'I-ORG':
      soccer_team[i][0] = soccer_team[i][0] + ' '+ soccer_team[i+1][0]
      soccer_team.pop(i+1)
    if soccer_team[i][1] == 'B-LOC' and soccer_team[i+1][1] == 'I-LOC':
      if soccer_team[i][0][len(soccer_team[i][0])-1] == '.':
        soccer_team[i][0] = soccer_team[i][0] +soccer_team[i+1][0]
      else:  
        soccer_team[i][0] = soccer_team[i][0] + ' ' +soccer_team[i+1][0]
      soccer_team.pop(i+1)
    i+=1
  i = 0
  while i < len(soccer_team)-1:
    j = i+1
    while j < len(soccer_team):
      if (soccer_team[i][0] != soccer_team[j][0]):
        if (soccer_team[i][0] in soccer_team[j][0]) or (soccer_team[j][0] in soccer_team[i][0]):
          if (soccer_team.count(soccer_team[i]) > soccer_team.count(soccer_team[j])) :
            t = soccer_team[j]
            soccer_team = [x for x in soccer_team if x!= t ]
          else :
            t = soccer_team[i]
            soccer_team = [x for x in soccer_team if x!= t ]
          i = 0
          j = i+1
          continue      
      j+=1
    i+=1
  trash_can = []
  i = 0
  while i < len(soccer_team):
    if soccer_team[i][0] in trash_can:
      i+=1
      continue
    if soccer_team[i][0][0].islower():
      t = soccer_team[i]
      soccer_team = [x for x in soccer_team if x!= t]
      trash_can.append(t)
    i+=1
  listA = []
  for x in soccer_team:
    listA.append(tuple(x))
  #print("Given List:\n",listA)
  team_occurence_count = Counter(listA)
  team1=team_occurence_count.most_common(2)[0][0][0]
  team1 = team1.replace('_',' ')
  team2 = team_occurence_count.most_common(2)[1][0][0]
  team2 = team2.replace('_',' ')
  d = {"test_id": "%d"%(test_id), "match_summary": {"players": {"team1": "%s"%team1, "team2": "%s"%team2}, "score_board": {"score1": "0", "score2": "0"}, "score_list": [{"player_name": "", "time": "", "team": ""}], "card_list": [{"player_name": "", "time": "", "team": ""}], "substitution_list": [{"player_in": "", "time": "", "player_out": ""}]}}
  list_dict.append(d)
  ##print(d)
  

[[('Pha', 'O'), ('sút', 'O'), ('bóng', 'O'), ('của', 'O'), ('cầu_thủ', 'O'), ('Sanna_Khánh', 'B-PER'), ('Hoà-BVN', 'I-PER'), ('(', 'O'), ('quần_áo', 'O'), ('xanh', 'O'), (')', 'O'), ('.', 'O')], [('(', 'O'), ('Ảnh', 'O'), (':', 'O'), ('Tiên_Minh', 'B-PER'), ('/', 'O'), ('TTXVN', 'B-ORG'), (')', 'O'), ('Chiều', 'O'), ('12/2', 'O'), (',', 'O'), ('trên', 'O'), ('sân', 'O'), ('19/8', 'O'), (',', 'O'), ('thành_phố', 'B-LOC'), ('Nha_Trang', 'I-LOC'), ('(', 'O'), ('Khánh_Hoà', 'B-LOC'), (')', 'O'), (',', 'O'), ('đội', 'O'), ('bóng', 'O'), ('chủ', 'O'), ('nhà', 'O'), ('Sanna_Khánh', 'O'), ('Hoà-BVN', 'B-PER'), ('đã', 'O'), ('không_thể', 'O'), ('chống_đỡ', 'O'), ('những', 'O'), ('chân', 'O'), ('sút', 'O'), ('đầy', 'O'), ('hiệu_quả', 'O'), ('của', 'O'), ('Than_Quang_Ninh', 'B-PER'), (',', 'O'), ('chấp_nhận', 'O'), ('thua', 'O'), ('đậm', 'O'), ('0-3', 'O'), ('sau', 'O'), ('khi', 'O'), ('tiếng', 'O'), ('còi', 'O'), ('kết_thúc', 'O'), ('trận', 'O'), ('đấu', 'O'), ('của', 'O'), ('trọng_tài', 'O'), (

WRITE

In [ ]:
with open('submissions-jsonl', 'w') as out_file:
    for i in range(len(list_dict)):
      out_file.write(json.dumps(list_dict[i],ensure_ascii=False))
      out_file.write("\n")

time: 2.9 ms


In [ ]:
players_name = [x for x in ner_merge if x[1] == 'B-PER' or x[1] =='I-PER']
players_name_occurence_count = Counter(players_name)
print(players_name_occurence_count.most_common(5))

In [ ]:
vncorenlp.close()

SCORE

In [ ]:
sub = pd.read_json("/content/submissions-jsonl",lines = True)

time: 18.6 ms


In [ ]:
list_dict = []
for i in range(1):
  test_id = test['test_id'][i]
  team1 = sub['match_summary'][i]['players']['team1']
  team2 = sub['match_summary'][i]['players']['team2']
  text = test['original_doc'][i]
  test_id = test['test_id'][i]
  size_temp = len(text["_source"]["body"])
  sentences = [] 
  for i in range(size_temp):
    temp = ((text["_source"]["body"][i]["text"]))
    sentences.append(temp)
  s = str()
  s = s + ' ' + ' '.join(sentences)
  s = s + '. ' + str(text["_source"]["description"])
  clean_text = s
  scores =  []
  for j in range(len(clean_text)):
    if clean_text[j] == '-' and clean_text[j+2]!='-' and clean_text[j-2]!='-':
      if clean_text[j-1].isnumeric() and clean_text[j+1].isnumeric():
        scores.append(clean_text[j-1]+clean_text[j]+clean_text[j+1])
    if clean_text[j] == '-' and clean_text[j-2].isnumeric() and clean_text[j+2].isnumeric():
      scores.append(clean_text[j-2]+clean_text[j]+clean_text[j+2])

    if clean_text[j] == '–' and clean_text[j+2]!='–' and clean_text[j-2]!='–':
      if clean_text[j-1].isnumeric() and clean_text[j+1].isnumeric():
        scores.append(clean_text[j-1]+clean_text[j]+clean_text[j+1])
    if clean_text[j] == '–' and clean_text[j-2].isnumeric() and clean_text[j+2].isnumeric():
      scores.append(clean_text[j-2]+clean_text[j]+clean_text[j+2])
  listA = scores
  print(scores)
  #print("Given List:\n",listA)
  if (len(listA) == 0):
    final_score_1 = 0
    final_score_2 = 0
  else :
    scores_occurence_count = Counter(listA)
    #print(team_occurence_count.most_common(2))
    final_score_1 = scores_occurence_count.most_common(1)[0][0][0]
    final_score_2 = scores_occurence_count.most_common(1)[0][0][2]
  d = {"test_id": "%d"%(test_id), "match_summary": {"players": {"team1": "%s"%team1, "team2": "%s"%team2}, "score_board": {"score1": "%s"%final_score_1, "score2": "%s"%final_score_2}, "score_list": [{"player_name": "", "time": "", "team": ""}], "card_list": [{"player_name": "", "time": "", "team": ""}], "substitution_list": [{"player_in": "", "time": "", "player_out": ""}]}}
  list_dict.append(d)
  print(d)

['1-0', '3-0', '1-3', '3-1', '0-3']
{'test_id': '21464024', 'match_summary': {'players': {'team1': 'Than Quảng Ninh', 'team2': 'TTXVN'}, 'score_board': {'score1': '1', 'score2': '0'}, 'score_list': [{'player_name': '', 'time': '', 'team': ''}], 'card_list': [{'player_name': '', 'time': '', 'team': ''}], 'substitution_list': [{'player_in': '', 'time': '', 'player_out': ''}]}}
time: 51.3 ms
